# Heat Recovery Calculations

## Thermal resistance model

### Surface tempreture


In [4]:
# --- Cell 1: helpers ---
import math

def conv_resistance(h, A):
    """
    Convection resistance: R = 1/(h*A)
    Returns (R [K/W], UA [W/K])
    """
    if h <= 0 or A <= 0:
        raise ValueError("h and A must be positive.")
    UA = h * A
    return 1.0 / UA, UA

def cond_resistance(k, A, L):
    """
    Conduction through a slab: R = L/(k*A)
    Returns (R [K/W], UA [W/K])
    """
    if k <= 0 or A <= 0 or L <= 0:
        raise ValueError("k, A, L must be positive.")
    UA = k * A / L
    return 1.0 / UA, UA


In [5]:
# --- Cell 2: solver ---
def solve_pt1000_no_hex(
    T_air_hot_C,
    h_hot, A_hot,
    k_wall, A_wall, wall_thickness_m,
    adiabatic_back=True,
    T_back_ambient_C=25.0,
    h_back=5.0,
    A_back=None
):
    """
    Returns a dict with q_W, node_temps_C, resistances_K_per_W, UA_W_per_K.
    If adiabatic_back=True, no heat leaves the back (ideal 'no heat recovered').
    """
    if A_back is None:
        A_back = A_wall

    # Hot side convection
    R_hot, UA_hot = conv_resistance(h_hot, A_hot)

    # Wall conduction
    R_wall, UA_wall = cond_resistance(k_wall, A_wall, wall_thickness_m)

    if adiabatic_back:
        # Physically: q -> 0 at steady state, surface tends to hot air temperature
        q = 0.0
        T_surface = T_air_hot_C
        T_back_wall = T_surface
        R_back = float('inf')
        UA_back = 0.0
        R_total = float('inf')
        UA_total = 0.0
        return {
            'q_W': q,
            'node_temps_C': {
                'T_air_hot': T_air_hot_C,
                'T_surface_PT1000': T_surface,
                'T_back_wall': T_back_wall,
                'T_back_fluid': T_back_ambient_C,
            },
            'resistances_K_per_W': {
                'R_hot': R_hot,
                'R_wall': R_wall,
                'R_back': R_back,
                'R_total': R_total,
            },
            'UA_W_per_K': {
                'UA_hot': UA_hot,
                'UA_wall': UA_wall,
                'UA_back': UA_back,
                'UA_total': UA_total,
            }
        }
    else:
        # Weak back convection to ambient
        R_back, UA_back = conv_resistance(h_back, A_back)
        R_total = R_hot + R_wall + R_back
        if not math.isfinite(R_total) or R_total <= 0:
            raise ValueError("Total resistance must be finite and positive.")
        q = (T_air_hot_C - T_back_ambient_C) / R_total

        # Node temperatures
        T_surface = T_air_hot_C - q * R_hot
        T_back_wall = T_surface - q * R_wall
        T_back_fluid = T_back_wall - q * R_back

        return {
            'q_W': q,
            'node_temps_C': {
                'T_air_hot': T_air_hot_C,
                'T_surface_PT1000': T_surface,
                'T_back_wall': T_back_wall,
                'T_back_fluid': T_back_fluid,
            },
            'resistances_K_per_W': {
                'R_hot': R_hot,
                'R_wall': R_wall,
                'R_back': R_back,
                'R_total': R_total,
            },
            'UA_W_per_K': {
                'UA_hot': UA_hot,
                'UA_wall': UA_wall,
                'UA_back': UA_back,
                'UA_total': 1.0 / R_total,
            }
        }


In [6]:
# --- Cell 3: quick test ---
T_air_hot = 300.0   # °C
h_hot = 120.0       # W/m²K (forced convection)
A = 0.02            # m²
k_wall = 16.0       # W/m·K (stainless-like)
t_wall = 0.004      # m

print("Case A: adiabatic back (no heat recovered)")
resA = solve_pt1000_no_hex(
    T_air_hot_C=T_air_hot,
    h_hot=h_hot, A_hot=A,
    k_wall=k_wall, A_wall=A, wall_thickness_m=t_wall,
    adiabatic_back=True
)
print("PT1000 surface (°C):", resA['node_temps_C']['T_surface_PT1000'])
print("q (W):", resA['q_W'])

print("\nCase B: weak back convection (h_back=5 W/m²K to 25°C)")
resB = solve_pt1000_no_hex(
    T_air_hot_C=T_air_hot,
    h_hot=h_hot, A_hot=A,
    k_wall=k_wall, A_wall=A, wall_thickness_m=t_wall,
    adiabatic_back=False,
    T_back_ambient_C=25.0,
    h_back=5.0,
    A_back=A
)
print("PT1000 surface (°C):", resB['node_temps_C']['T_surface_PT1000'])
print("q (W):", resB['q_W'])

Case A: adiabatic back (no heat recovered)
PT1000 surface (°C): 300.0
q (W): 0.0

Case B: weak back convection (h_back=5 W/m²K to 25°C)
PT1000 surface (°C): 289.0131841789852
q (W): 26.368357970435476


### Tempreture when heat recovered

In [2]:
"""
Thermal Resistance Network with Optional Finned Heat Exchanger
Purpose: Estimate surface/base temperature and heat rate between two fluids across a wall,
         including contact resistance and a finned heat exchanger on one side.

Units:
  - Temperatures in °C (internally converted to K for differences)
  - h in W/(m^2·K)
  - k in W/(m·K)
  - Areas in m^2
  - Lengths in m
  - R in K/W
  - q in W
"""

import math
from dataclasses import dataclass
from typing import Dict, Tuple, Optional

# ---------------------------
# Basic Resistances (building blocks)
# ---------------------------

def conv_resistance(h: float, area: float) -> Tuple[float, float]:
    """
    Convection resistance: R = 1 / (h * A)
    Returns: (R_conv [K/W], UA [W/K])
    """
    if h <= 0 or area <= 0:
        raise ValueError("h and area must be positive.")
    UA = h * area
    R = 1.0 / UA
    return R, UA

def cond_resistance(k: float, area: float, thickness: float) -> Tuple[float, float]:
    """
    Conduction resistance through a slab: R = L / (k * A)
    Returns: (R_cond [K/W], UA [W/K])
    """
    if k <= 0 or area <= 0 or thickness <= 0:
        raise ValueError("k, area, and thickness must be positive.")
    UA = k * area / thickness
    R = 1.0 / UA
    return R, UA

def contact_resistance_from_h(h_contact: float, area: float) -> Tuple[float, float]:
    """
    Thermal contact resistance modeled as a contact conductance h_c over area A:
    R_contact = 1 / (h_c * A)
    Returns: (R_contact [K/W], UA_contact [W/K])
    """
    return conv_resistance(h_contact, area)

def contact_resistance_from_R(R_contact: float) -> Tuple[float, float]:
    """
    Use a provided contact resistance directly.
    Returns: (R_contact [K/W], UA_contact [W/K])
    """
    if R_contact <= 0:
        raise ValueError("R_contact must be positive.")
    return R_contact, 1.0 / R_contact


# ---------------------------
# Fins and Effective Convection on HEX side
# ---------------------------

def fin_efficiency_rectangular(k_fin: float, h: float, t_fin: float, L_fin: float) -> float:
    """
    Straight rectangular fin efficiency (adiabatic tip approximation):
    eta_f = tanh(m L) / (m L), where m = sqrt(h * P / (k * A_c))
    For a thin plate fin of thickness t and width w (into page), P ≈ 2w, A_c = t*w,
    so m ≈ sqrt(2h / (k * t)) (independent of w).
    """
    if min(k_fin, h, t_fin, L_fin) <= 0:
        raise ValueError("k_fin, h, t_fin, L_fin must be positive.")
    m = math.sqrt(2.0 * h / (k_fin * t_fin))
    mL = m * L_fin
    if mL == 0:
        return 1.0
    eta_f = math.tanh(mL) / mL
    return eta_f

def overall_surface_efficiency(eta_f: float, A_fin: float, A_total: float) -> float:
    """
    Overall surface (surface/fin) efficiency:
    eta_o = 1 - (A_f / A_t) * (1 - eta_f)
      A_fin   = total fin area exposed to convection (sides + tips, exclude base)
      A_total = A_base_exposed + A_fin
    """
    if A_total <= 0 or A_fin < 0:
        raise ValueError("Areas must be A_total > 0 and A_fin >= 0.")
    if eta_f < 0 or eta_f > 1:
        raise ValueError("eta_f must be in [0, 1].")
    return 1.0 - (A_fin / A_total) * (1.0 - eta_f)

def effective_conv_with_fins(h: float, A_total: float, A_fin: float, eta_f: float) -> Tuple[float, float, float]:
    """
    Effective convection with fins:
      UA_eff = h * [A_base_exposed + eta_f * A_fin] = h * A_total * eta_o
      R_eff = 1 / UA_eff
    Returns: (R_conv_eff [K/W], UA_eff [W/K], eta_o)
    """
    eta_o = overall_surface_efficiency(eta_f, A_fin, A_total)
    UA_eff = h * A_total * eta_o
    if UA_eff <= 0:
        raise ValueError("Effective UA must be positive. Check inputs.")
    R_eff = 1.0 / UA_eff
    return R_eff, UA_eff, eta_o
# ---------------------------
# Optional: Simple h estimators (use cautiously; prefer measured h)
# ---------------------------

def h_external_flow_flat_plate(velocity: float, length: float,
                               rho: float, mu: float, k: float, Pr: float) -> float:
    """
    Very simplified average h for turbulent external flow over a flat plate (isothermal):
    Nu_L ≈ 0.037 * Re_L^0.8 * Pr / (1 + 2.443 * Re_L^{-0.1} * (Pr^{2/3} - 1))
    h = Nu_L * k / L
    NOTE: This is a rough estimator. Validate against your geometry/flow regime.
    """
    if min(velocity, length, rho, mu, k, Pr) <= 0:
        raise ValueError("All inputs must be positive.")
    Re = rho * velocity * length / mu
    # Guard for laminar regime—this correlation is primarily for turbulent; for low Re, return small h
    if Re < 5e5:
        Nu = 0.664 * math.sqrt(Re) * (Pr ** (1/3))
    else:
        Nu = 0.037 * (Re ** 0.8) * Pr / (1.0 + 2.443 * (Re ** -0.1) * ((Pr ** (2/3)) - 1.0))
    h = Nu * k / length
    return h
# ---------------------------
# Solver and reporting
# ---------------------------

@dataclass
class ThermalResult:
    q_W: float
    node_temps_C: Dict[str, float]
    resistances_K_per_W: Dict[str, float]
    deltaT_contributions_K: Dict[str, float]
    deltaT_shares_pct: Dict[str, float]
    UA_summary_W_per_K: Dict[str, float]
    eta_o_cold_side: Optional[float] = None

def solve_two_sided_wall(
    T_hot_C: float, T_cold_C: float,
    # Hot side convection
    h_hot: float, A_hot: float,
    # Wall conduction
    k_wall: float, A_wall: float, wall_thickness: float,
    # Contact between wall and HEX base (optional: set to 0 for perfect)
    R_contact: float = 0.0, h_contact: Optional[float] = None, A_contact: Optional[float] = None,
    # Cold side convection (with optional fins)
    h_cold: float = 10.0,
    A_cold_total: float = 1.0,
    A_cold_fin: float = 0.0,
    eta_f: Optional[float] = None,  # provide if using fins; if None and A_cold_fin>0, raise
) -> ThermalResult:
    """
    Build the series thermal network and solve for:
      - Heat rate q
      - Node temperatures: hot fluid, hot wall, cold wall/base, cold fluid
      - Contribution of each resistance to the total temperature drop

    If A_cold_fin > 0 you must pass eta_f, or precompute and pass here.
    """
    # Hot side convection
    R_hot, UA_hot = conv_resistance(h_hot, A_hot)

    # Wall conduction
    R_wall, UA_wall = cond_resistance(k_wall, A_wall, wall_thickness)

    # Contact
    if R_contact > 0 and h_contact is not None:
        raise ValueError("Specify either R_contact or (h_contact & A_contact), not both.")
    if h_contact is not None:
        if A_contact is None:
            raise ValueError("Provide A_contact with h_contact.")
        R_cnt, UA_cnt = contact_resistance_from_h(h_contact, A_contact)
    else:
        R_cnt, UA_cnt = (0.0, float('inf')) if R_contact == 0 else contact_resistance_from_R(R_contact)

    # Cold side convection (with or without fins)
    if A_cold_fin > 0:
        if eta_f is None:
            raise ValueError("A_cold_fin > 0 but eta_f not provided. Compute fin efficiency and pass it.")
        R_cold, UA_cold, eta_o = effective_conv_with_fins(h_cold, A_cold_total, A_cold_fin, eta_f)
    else:
        R_cold, UA_cold = conv_resistance(h_cold, A_cold_total)
        eta_o = None

    # Total network
    R_total = R_hot + R_wall + R_cnt + R_cold
    deltaT_total = (T_hot_C - T_cold_C)
    if R_total <= 0:
        raise ValueError("Total resistance must be positive.")
    q = deltaT_total / R_total  # W

    # Node temperatures (from hot to cold)
    T_hot_fluid = T_hot_C
    T_hot_wall = T_hot_fluid - q * R_hot
    T_cold_wall = T_hot_wall - q * R_wall
    T_base_under_hex = T_cold_wall - q * R_cnt
    T_cold_fluid = T_base_under_hex - q * R_cold

    # Sanity check
    # Allow small numerical deviation
    if abs(T_cold_fluid - T_cold_C) > max(1e-6, 1e-3 * abs(deltaT_total)):
        # Not raising—just note: rounding may cause tiny differences
        pass

    # Contributions
    dT_hot = q * R_hot
    dT_wall = q * R_wall
    dT_cnt = q * R_cnt
    dT_cold = q * R_cold
    shares = {
        "R_hot_conv": 100.0 * dT_hot / deltaT_total if deltaT_total != 0 else 0.0,
        "R_wall_cond": 100.0 * dT_wall / deltaT_total if deltaT_total != 0 else 0.0,
        "R_contact": 100.0 * dT_cnt / deltaT_total if deltaT_total != 0 else 0.0,
        "R_cold_conv": 100.0 * dT_cold / deltaT_total if deltaT_total != 0 else 0.0,
    }

    return ThermalResult(
        q_W=q,
        node_temps_C={
            "T_hot_fluid": T_hot_fluid,
            "T_hot_wall": T_hot_wall,
            "T_cold_wall": T_cold_wall,
            "T_base_under_hex": T_base_under_hex,
            "T_cold_fluid": T_cold_fluid,
        },
        resistances_K_per_W={
            "R_hot_conv": R_hot,
            "R_wall_cond": R_wall,
            "R_contact": R_cnt,
            "R_cold_conv": R_cold,
            "R_total": R_total,
        },
        deltaT_contributions_K={
            "dT_hot_conv": dT_hot,
            "dT_wall_cond": dT_wall,
            "dT_contact": dT_cnt,
            "dT_cold_conv": dT_cold,
        },
        deltaT_shares_pct=shares,
        UA_summary_W_per_K={
            "UA_hot": UA_hot,
            "UA_wall": UA_wall,
            "UA_contact": UA_cnt,
            "UA_cold": UA_cold,
            "UA_total": 1.0 / R_total,
        },
        eta_o_cold_side=eta_o,
    )


# ---------------------------
# Example usage (fill in with your values)
# ---------------------------

if __name__ == "__main__":
    # Example scenario:
    # Hot side: high-velocity hot air at 300°C
    T_hot = 300.0  # °C
    # Cold side fluid (e.g., ambient air or coolant) at 25°C
    T_cold = 25.0  # °C

    # Hot-side convection area and h (estimate or measured)
    h_hot = 120.0      # W/m^2·K  (example for high-speed air)
    A_hot = 0.02       # m^2

    # Wall
    k_wall = 16.0      # W/m·K (e.g., stainless steel ~15-20 W/m·K)
    A_wall = 0.02      # m^2 (same footprint)
    t_wall = 0.004     # m (4 mm)

    # Contact between wall and HEX base
    # Option 1: specify R_contact directly (e.g., with thermal pad)
    R_contact = 0.02   # K/W  (example)
    # Option 2: specify contact conductance instead of R_contact
    h_contact = None
    A_contact = None

    # Cold side with fins (heat exchanger)
    h_cold = 30.0        # W/m^2·K (example: natural to forced convection)
    A_cold_base = 0.02   # m^2
    # Fin geometry (example): compute eta_f and areas externally or just set A_fin and eta_f
    k_fin = 200.0        # W/m·K (Aluminum)
    t_fin = 0.0015       # 1.5 mm
    L_fin = 0.02         # 20 mm fin length
    eta_f = fin_efficiency_rectangular(k_fin=k_fin, h=h_cold, t_fin=t_fin, L_fin=L_fin)

    # Suppose we have N fins of width W into page; compute fin area (sides + tips):
    N_fins = 20
    W = 0.02             # m (fin width into page)
    A_fin_single = 2 * L_fin * W + t_fin * W    # side area + tip area
    A_fin_total = N_fins * A_fin_single

    # Base exposed area after fins occupy some base footprint:
    A_base_covered = N_fins * (t_fin * W)
    A_base_exposed = max(A_cold_base - A_base_covered, 0.0)

    A_cold_total = A_base_exposed + A_fin_total

    result = solve_two_sided_wall(
        T_hot_C=T_hot, T_cold_C=T_cold,
        h_hot=h_hot, A_hot=A_hot,
        k_wall=k_wall, A_wall=A_wall, wall_thickness=t_wall,
        R_contact=R_contact,  # or h_contact=..., A_contact=...
        h_cold=h_cold, A_cold_total=A_cold_total, A_cold_fin=A_fin_total, eta_f=eta_f
    )

    print("\n=== RESULTS ===")
    print(f"Heat rate q = {result.q_W:.2f} W")
    for k, v in result.node_temps_C.items():
        print(f"{k}: {v:.2f} °C")
    print("\nResistances [K/W]:")
    for k, v in result.resistances_K_per_W.items():
        print(f"{k}: {v:.6f}")
    print("\nΔT contributions [K]:")
    for k, v in result.deltaT_contributions_K.items():
        print(f"{k}: {v:.2f}")
    print("\nΔT shares [%]:")
    for k, v in result.deltaT_shares_pct.items():
        print(f"{k}: {v:.1f}%")
    print("\nUA Summary [W/K]:")
    for k, v in result.UA_summary_W_per_K.items():
        print(f"{k}: {v:.2f}")
    if result.eta_o_cold_side is not None:
        print(f"\nOverall surface efficiency on HEX side η_o = {result.eta_o_cold_side:.3f}")


=== RESULTS ===
Heat rate q = 198.38 W
T_hot_fluid: 300.00 °C
T_hot_wall: 217.34 °C
T_cold_wall: 214.86 °C
T_base_under_hex: 210.90 °C
T_cold_fluid: 25.00 °C

Resistances [K/W]:
R_hot_conv: 0.416667
R_wall_cond: 0.012500
R_contact: 0.020000
R_cold_conv: 0.937092
R_total: 1.386258

ΔT contributions [K]:
dT_hot_conv: 82.66
dT_wall_cond: 2.48
dT_contact: 3.97
dT_cold_conv: 185.90

ΔT shares [%]:
R_hot_conv: 30.1%
R_wall_cond: 0.9%
R_contact: 1.4%
R_cold_conv: 67.6%

UA Summary [W/K]:
UA_hot: 2.40
UA_wall: 80.00
UA_contact: 50.00
UA_cold: 1.07
UA_total: 0.72

Overall surface efficiency on HEX side η_o = 0.988
